In [49]:
import pandas as pd
df = pd.read_csv('train.csv')
li = df[df.isnull().T.any()].index
for i in li:
    df.drop(index=i,axis = 1,inplace = True)
df[df.isnull().T.any()]
df

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive
...,...,...,...,...
27481,3dbae74fcd,"I want to go to VP, but no one is willing to c...","I want to go to VP, but no one is willing to c...",neutral
27482,63147b35cb,"Wah, why are you sad?","Wah, why are you sad?",neutral
27483,bdb196a09f,playing sudoku while mommy makes me breakfast ...,playing sudoku while mommy makes me breakfast ...,neutral
27484,18c2a1e98e,see u bye see u! i love the hot30,i love,positive


In [50]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer

def custom_preprocessor(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
#     words = word_tokenize(text)
#     stemmer = PorterStemmer()
#     wnl = WordNetLemmatizer()
#     words = [wnl.lemmatize(i) for i in words if i not in stopwords.words()]
#     text = ' '.join(words)
    
    return text

In [51]:
df['polarity_new']=0
df.loc[df['sentiment']=='negative',['polarity_new']]=-1
df.loc[df['sentiment']=='neutral',['polarity_new']]=0
df.loc[df['sentiment']=='positive',['polarity_new']]=1
df['polarity_new'].value_counts()

 0    11117
 1     8582
-1     7786
Name: polarity_new, dtype: int64

In [52]:
df['text_clean']=df['selected_text'].apply(custom_preprocessor)

In [18]:
df.to_csv('text_clean.csv',index=False)

In [1]:
import pandas as pd
df = pd.read_csv('text_clean.csv')
df

,textID,text,selected_text,sentiment,polarity_new,text_clean
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral,0,bos happy w lot fun
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive,1,good
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral,0,say good say bad afternoon
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative,-1,dont think vote anymore
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive,1,better
...,...,...,...,...,...,...
27480,3dbae74fcd,"I want to go to VP, but no one is willing to c...","I want to go to VP, but no one is willing to c...",neutral,0,go vp willing
27481,63147b35cb,"Wah, why are you sad?","Wah, why are you sad?",neutral,0,sad
27482,bdb196a09f,playing sudoku while mommy makes me breakfast ...,playing sudoku while mommy makes me breakfast ...,neutral,0,playing sudoku mommy make breakfast amp lun
27483,18c2a1e98e,see u bye see u! i love the hot30,i love,positive,1,love


In [ ]:
df1 = 

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

In [66]:
X = df['text_clean']
Y = df['polarity_new']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
X_train.shape

(19239,)

In [67]:
from sklearn.feature_extraction.text import TfidfTransformer 
vect = CountVectorizer(ngram_range=(1,2),max_features=2000)
transformer = TfidfTransformer()
X_train_dtm = vect.fit_transform(X_train.astype('U'))
X_train_dtm = transformer.fit_transform(X_train_dtm)
X_test_dtm = vect.transform(X_test.astype('U'))
X_test_dtm = transformer.transform(X_test_dtm)
X_test_dtm

<8246x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 24762 stored elements in Compressed Sparse Row format>

## 逻辑回归

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# instantiate a logistic regression model
logreg = LogisticRegression(multi_class='multinomial', solver="lbfgs", C=10, random_state=123)

In [69]:
%time logreg.fit(X_train_dtm,y_train)

CPU times: user 1.1 s, sys: 17.4 ms, total: 1.12 s
Wall time: 584 ms


/Users/linqiaoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, multi_class='multinomial', random_state=123)

In [70]:
from sklearn.metrics import classification_report
y_test_pred = logreg.predict(X_test_dtm)
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))
print(classification_report(y_test,y_test_pred))

Accuracy:0.7838952219257822
Precision:0.7838952219257822
Recall:0.7838952219257822
              precision    recall  f1-score   support

          -1       0.71      0.78      0.75      2378
           0       0.80      0.80      0.80      3314
           1       0.84      0.76      0.80      2554

    accuracy                           0.78      8246
   macro avg       0.78      0.78      0.78      8246
weighted avg       0.79      0.78      0.78      8246



In [71]:
import numpy as np
df1 = pd.read_csv('Finished_vaccination_tweets_手动标记.csv',encoding='ISO-8859-1', error_bad_lines=False)
df1 = df1[(df1['code-attitude'] == '0') | (df1['code-attitude'] == '1') | (df1['code-attitude'] == '-1')]
df1['text']

2       Facts are immutable, Senator, even when you're...
3       Explain to me again why we need a vaccine @Bor...
4       Does anyone have any useful advice/guidance fo...
5       it is a bit sad to claim the fame for success ...
6       There have not been many bright days in 2020 b...
                              ...                        
3101    #PublicHealth #COVID19 #Modernavaccine #Modern...
3102    @crashoverrideee #COVID19 Vaccine Update for #...
3103    Dr Fun's I Feel Good - My effort at a PSA, ple...
3104    @Writer_DG I got dose #2 Friday. Have to admit...
3105    Feeling very privileged to have had my first c...
Name: text, Length: 247, dtype: object

In [72]:
df1['text'] = pd.DataFrame(df1['text'].astype(str))
df1['text_clean']=df1['text'].apply(custom_preprocessor)
t = vect.transform(df1['text'])
y_test_pred = logreg.predict(t)
df1['code-attitude'] = pd.DataFrame(df1['code-attitude'].astype(int))
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred)))
print(classification_report(df1['code-attitude'],y_test_pred))

Accuracy:0.4817813765182186
              precision    recall  f1-score   support

          -1       0.44      0.12      0.19        58
           0       0.49      0.74      0.59       113
           1       0.46      0.37      0.41        76

    accuracy                           0.48       247
   macro avg       0.46      0.41      0.40       247
weighted avg       0.47      0.48      0.44       247



## 决策树

In [73]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)
clf.fit(X_train_dtm,y_train)
from sklearn.metrics import classification_report
y_test_pred = clf.predict(X_test_dtm)
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))
print(classification_report(y_test,y_test_pred))

Accuracy:0.5323793354353626
Precision:0.5323793354353626
Recall:0.5323793354353626
              precision    recall  f1-score   support

          -1       0.93      0.12      0.21      2378
           0       0.47      0.98      0.63      3314
           1       0.89      0.34      0.49      2554

    accuracy                           0.53      8246
   macro avg       0.76      0.48      0.44      8246
weighted avg       0.73      0.53      0.47      8246



## 随机森林

In [74]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, n_estimators=300)
rf.fit(X_train_dtm, y_train)
from sklearn.metrics import classification_report
y_test_pred = rf.predict(X_test_dtm)
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))
print(classification_report(y_test,y_test_pred))

Accuracy:0.7752849866601988
Precision:0.7752849866601988
Recall:0.7752849866601988
              precision    recall  f1-score   support

          -1       0.72      0.75      0.73      2378
           0       0.77      0.82      0.79      3314
           1       0.85      0.73      0.79      2554

    accuracy                           0.78      8246
   macro avg       0.78      0.77      0.77      8246
weighted avg       0.78      0.78      0.78      8246



In [75]:
df1['text'] = pd.DataFrame(df1['text'].astype(str))
df1['text_clean']=df1['text'].apply(custom_preprocessor)
t = vect.transform(df1['text'])
y_test_pred_logreg = logreg.predict(t)
y_test_pred_clf = clf.predict(t)
y_test_pred_rf = rf.predict(t)
df1['code-attitude'] = pd.DataFrame(df1['code-attitude'].astype(int))

print("logistic regression Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred_logreg)))
print("decision tree Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred_clf)))
print("random forest Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred_rf)))
print(classification_report(df1['code-attitude'],y_test_pred_logreg))
print(classification_report(df1['code-attitude'],y_test_pred_clf))
print(classification_report(df1['code-attitude'],y_test_pred_rf))

logistic regression Accuracy:0.4817813765182186
decision tree Accuracy:0.5263157894736842
random forest Accuracy:0.4979757085020243
              precision    recall  f1-score   support

          -1       0.44      0.12      0.19        58
           0       0.49      0.74      0.59       113
           1       0.46      0.37      0.41        76

    accuracy                           0.48       247
   macro avg       0.46      0.41      0.40       247
weighted avg       0.47      0.48      0.44       247

              precision    recall  f1-score   support

          -1       0.50      0.02      0.03        58
           0       0.50      0.98      0.66       113
           1       0.86      0.24      0.37        76

    accuracy                           0.53       247
   macro avg       0.62      0.41      0.35       247
weighted avg       0.61      0.53      0.42       247

              precision    recall  f1-score   support

          -1       0.35      0.33      0.34        